In [1]:
!pip install ftfy regex tqdm torch
!pip install git+https://github.com/openai/CLIP.git
!pip install pytorch-lightning

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-sj13j6n8
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-sj13j6n8
  Resolved https://github.com/openai/CLIP.git to commit b46f5ac7587d2e1862f8b7b1573179d80dcdd620


In [28]:
import clip
import torch
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from sklearn.metrics import top_k_accuracy_score
from sklearn.model_selection import train_test_split
import pickle
from tqdm import tqdm
import gc
import os
import pickle
from aliproduct_model import ALIPRODUCT_CLIP
from dataset import ALIPRODUCT_DATASET,prepare_data
from CONFIG import CONFIG
import faiss

In [3]:
!nvidia-smi

Tue May  3 21:37:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.60.02    Driver Version: 510.60.02    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA TITAN X ...  Off  | 00000000:02:00.0 Off |                  N/A |
| 30%   55C    P0    80W / 250W |      8MiB / 12288MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA TITAN X ...  Off  | 00000000:03:00.0 Off |                  N/A |
| 23%   

In [4]:
# image_data_dir = "../../"
# image_data_folder = "val_imgs"
# df = pd.read_csv("../data/train_split_val_data_map.csv")
# df.head()
# df.shape

In [5]:
# x_train,x_test = train_test_split(df,random_state= 101,test_size=0.2)

In [6]:
# class AI_CITY_DATASET():
#     def __init__(self,df,dir,folder,preprocess,model=None,precompute_txt_embed=False):
#         self.df = df
#         self.dir = dir
#         self.folder = folder
#         self.preprocess = preprocess
#         self.model = model
#         self.z_shot_labels,self.z_shot_label_map = self.create_z_shot_labels(self.df["caption"].unique(),self.df["label"].unique())
#         self.precompute_txt_embed = precompute_txt_embed
#         if self.precompute_txt_embed:
#             self.z_shot_labels = self.precompute_labels(self.z_shot_labels)
        
#     def __len__(self):
#         return(len(self.df))
#     def __getitem__(self,idx):
#         image_name = self.df.iloc[idx,:]["full_path"]
#         image = Image.open(image_name).convert("RGB")
#         image = self.preprocess(image)
#         return image
    # def create_z_shot_labels(self,label_names,label):
    #     z_shot_labels = []
    #     z_shot_label_map  = {}
    #     for label_name, label, idx in zip(label_names, label, range(len(label))):
    #         z_shot_label = f"this is a {label_name}"
    #         z_shot_labels.append(z_shot_label)
    #         z_shot_label_map[idx] = {"label_name": label_name, "label": label}
    #     z_shot_labels = clip.tokenize(z_shot_labels,truncate=True)
    #     return z_shot_labels,z_shot_label_map
    # def precompute_labels(self,z_shot_labels):
    #     if os.path.exists("./precompute_label_embed.pkl"):
    #         filename = "./precompute_label_embed.pkl"
    #         file = open(filename,"rb")
    #         precomputed_labels = pickle.load(file)
    #         file.close()
    #         return precomputed_labels
    #     else:
    #         precomputed_labels = []
    #         for label in tqdm(z_shot_labels):
    #             label = label.to(CONFIG.device)
    #             embed = self.model.model.encode_text(torch.unsqueeze(label,0))
    #             precomputed_labels.append(embed.detach().cpu())
    #         filename = "./precompute_label_embed.pkl"
    #         file = open(filename,"wb")
    #         pickle.dump(torch.concat(precomputed_labels),file)
    #         file.close()
    #         return torch.concat(precomputed_labels)

            
        

In [7]:
clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14']

In [8]:
model,preprocess = clip.load(CONFIG.model_name,CONFIG.device)

In [9]:
test_loader,df = prepare_data(CONFIG.test_df_path,CONFIG.test_image_data_dir,CONFIG.test_image_data_folder
    ,CONFIG.test_image_col,CONFIG.test_label_col,CONFIG.batch_size,preprocess,CONFIG.global_random_state,test=True)

# z_shot_labels = data.z_shot_labels
# z_shot_label_map = data.z_shot_label_map


In [10]:
# class ALIPRODUCT_CLIP(pl.LightningModule):
#     def __init__(self,model):
#         super().__init__()
#         self.model = model

#     def forward(self,batch):
#         img= batch
#         image_features = self.model.encode_image(img)
#         # text_features = self.model.encode_text(label[0])
#         return image_features
#     def predict_step(self,batch,batch_idx):
#         with torch.no_grad():

#             image_features = self(batch)
#             text_features= z_shot_labels.to(CONFIG.device)
#             image_features /= image_features.norm(dim=-1, keepdim=True)
#             text_features /= text_features.norm(dim=-1, keepdim=True)
#             similarity = (100.0 * image_features @ text_features.T)

#             pred = similarity.float().softmax(dim=-1)
#             return pred


In [11]:
clip_model = ALIPRODUCT_CLIP(model)
checkpoint = torch.load("../aliproduct_run_1_ViT-B/32_val_clip_loss=0.85487.ckpt")
clip_model.load_state_dict(checkpoint["state_dict"])
trainer = Trainer(gpus=4,strategy="ddp")
pred = trainer.predict(clip_model,test_loader)


/home/ubuntu/anaconda3/lib/python3.9/site-packages/pytorch_lightning/loops/utilities.py:91: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Predicting DataLoader 0:   0%|          | 4/9762 [00:02<1:59:53,  1.36it/s]

libpng warning: iCCP: known incorrect sRGB profile


Predicting DataLoader 0:   0%|          | 24/9762 [00:07<52:32,  3.09it/s]  

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


...

Predicting DataLoader 0:  99%|█████████▉| 9710/9762 [2:07:10<00:40,  1.27it/s]

libpng warning: iCCP: known incorrect sRGB profile


Predicting DataLoader 0: 100%|█████████▉| 9746/9762 [2:07:37<00:12,  1.27it/s]

libpng warning: iCCP: known incorrect sRGB profile


Predicting DataLoader 0: 100%|██████████| 9762/9762 [2:07:47<00:00,  1.27it/s]


In [12]:
full_pred = tuple(map(torch.concat, zip(*pred)))
image_embed,text_embed = full_pred
image_embed_transpose = image_embed.T

In [29]:
faiss_index = faiss.IndexFlatIP(512)
faiss_index.add(image_embed.numpy().astype(np.float32))

In [14]:
top5_k_e,top5_k_y_pred = faiss_index.search(text_embed.numpy().astype(np.float32),5)
top10_k_e,top10_k_y_pred = faiss_index.search(text_embed.numpy().astype(np.float32),10)

In [15]:
top5_preds = np.array([1 if y_true in y_pred else 0 for y_true,y_pred in zip(df.index.values,top5_k_y_pred)])
top10_preds = np.array([1 if y_true in y_pred else 0 for y_true,y_pred in zip(df.index.values,top10_k_y_pred)])

In [16]:
top5_acc = top5_preds[top5_preds ==1].shape[0] / top5_preds.shape[0]
top10_acc = top10_preds[top10_preds ==1].shape[0] / top10_preds.shape[0]
comp_acc  = np.mean([top5_acc,top10_acc])

In [17]:
print("top 5 accuracy: "+str(top5_acc*100)+"%")
print("top 10 accuracy: "+str(top10_acc*100)+"%")
print("avg accuracy: "+str(comp_acc*100)+"%")


top 5 accuracy: 2.6464802422566063%
top 10 accuracy: 4.303396397615972%
avg accuracy: 3.4749383199362898%


In [18]:
# def topk_accuracy(y_true,pred,label_map,k):
#     pred_count = []
#     remap = np.vectorize(lambda x:label_map[x]["label"])
#     #get top k values 
#     top_k_pred = remap(pred.argsort(axis=1)[:,-k:][:,::-1])
#     # append 1 if true label is in top k values else append 0
#     for i in range(len(y_true)):
#         if y_true[i] in top_k_pred[i]:
#             pred_count.append(1)
#         else:
#             pred_count.append(0)
#     return sum(pred_count)/len(pred_count)


In [19]:
# print("top 5 accuracy: "+str(topk_accuracy(y_true,full_pred_np,z_shot_label_map,k=5)*100)+"%")
# print("top 10 accuracy: "+str(topk_accuracy(y_true,full_pred_np,z_shot_label_map,k=10)*100)+"%")

In [20]:
# top1 = full_pred.type(torch.float32).topk(1,dim=1)
# top5 = full_pred.type(torch.float32).topk(5,dim=1)
# top10 = full_pred.type(torch.float32).topk(10,dim=1)
# remap = np.vectorize(lambda x:z_shot_label_map[x]["label_name"])
# final_preds =list({l:p} for l,p in zip(remap(top1[1].flatten().tolist()),top1[0].flatten().tolist()))
# final_preds_top5 =list(dict(zip(remap(l),p)) for l,p in zip(top5[1].tolist(),top5[0].tolist()))
# final_preds_top10 =list(dict(zip(remap(l),p)) for l,p in zip(top10[1].tolist(),top10[0].tolist()))


In [21]:
# top5[1]

In [22]:
# preds_df = df
# preds_df["preds"] = final_preds
# preds_df["preds_top_5"] = final_preds_top5
# preds_df["preds_top_10"] = final_preds_top10
# preds_df.drop("Unnamed: 0",axis=1,inplace=True)

In [23]:
# preds_df.head()

In [24]:
# preds_df.to_csv("aicity4_all_pred.csv")

In [25]:
# filename = "al_pred.pkl"
# file = open(filename,"wb")
# pickle.dump(full_pred,file)
# file.close()

In [26]:
# #test laod pickle 
# file = open(filename,"rb")
# pkl_preds = pickle.load(file)
# file.close()

In [27]:
# pkl_preds